<a href="https://colab.research.google.com/github/aaubs/ds-master/blob/main/notebooks/M1-API-JSON-MongoDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Working with web- & app-data

In this session, you will learn to work with Python dictionaries, the JSON (Java Script Object Notation) format. This will also include dealing (a bit) with APIs and MongoDB

In [ ]:
# some necessary installs
!pip install "pymongo[srv]"
!pip install cloudscraper
!pip install srsly

In [ ]:
import pandas as pd

## Python dictionaries (recap)

Dictionaries are a super-flexible datatype that is able to take in and index nested structures.

In [ ]:
capitals = {"USA":"Washington D.C.", "France":"Paris", "India":"New Delhi"}

In [ ]:
capitals['Denmark'] = 'Copenhagen'

In [ ]:
capitals['Denmark']

In [ ]:
capitals.keys()

In [ ]:
capitals.values()

In [ ]:
country_info = {}

In [ ]:
country_info['USA'] = {'capital':"Washington D.C.", 'population': 329.5, 'cities': ["Washington D.C.",'New York','San Francisco']}

In [ ]:
country_info['Denmark'] = {'capital':"Copenhagen", 'population': 5.831, 'cities': ["Copenhagen",'Århus','Odense','Aalborg']}

In [ ]:
country_info

### Dicts in Pandas?

In [ ]:
pd.DataFrame(country_info)

In [ ]:
pd.DataFrame(country_info).T

In [ ]:
pd.DataFrame(country_info).T.to_json('country_info.json')

### Introducing srsly: Modern high-performance serialization utilities for Python

https://pypi.org/project/srsly/

In [ ]:
import srsly

In [ ]:
json_string = srsly.json_dumps(country_info)

In [ ]:
json_string

In [ ]:
srsly.write_json("country_info_str.json", country_info)

In [ ]:
country_info_from_str = srsly.read_json("country_info_str.json")

### Nomadlist API and Digital Nomads

We will look at https://nomadlist.com and its API to gather data about trips around the world by digital nomads.
* We will start with the `requests` library to perform simle API intraction.
* We will use the standard `json` library to parse the respons

To do so, we need to identify a user of the platform and then add `.json` to the URL

In [ ]:
# import the relevant libraries
import requests as rq
import json

In [ ]:
# get data
result = rq.get('https://nomadlist.com/@ambroisedebret.json')

In [ ]:
# parse the response content
doc = json.loads(result.content)

In [ ]:
handles = srsly.read_gzip_json("handles-out.json.gz")

In [ ]:
handles

In [ ]:
# alternatively
# handles = pd.read_json('/content/handles-out.json.gz', typ='series')

In [ ]:
# prepare handles for scrape
handle_list = pd.Series(handles).sample(10).to_list()

In [ ]:
# managing time...
import time

In [ ]:
profiles = []

In [ ]:
for i in handle_list:
  url = 'https://nomadlist.com' + i + '.json'
  result = rq.get(url)
  result_json = json.loads(result.content)
  profiles.append(result_json)
  time.sleep(1)

In [ ]:
result

Oh no!!! 🤯
Most modern webapps are protected against full-automatic scraping.
https://www.cloudflare.com/learning/bots/what-is-content-scraping/
However, it's a chicken and egg game...


**cloudscraper**

> A simple Python module to bypass Cloudflare's anti-bot page (also known as "I'm Under Attack Mode", or IUAM), implemented with Requests. Cloudflare changes their techniques periodically, so I will update this repo frequently.







In [ ]:
import cloudscraper

In [ ]:
# Instantiate scraper
scraper = cloudscraper.create_scraper()

In [ ]:
profiles = []

In [ ]:
for i in handle_list:
  url = 'https://nomadlist.com' + i + '.json'
  result = scraper.get(url)
  result_json = json.loads(result.content)
  profiles.append(result_json)
  time.sleep(1)

In [ ]:
len(profiles)

In [ ]:
trips = pd.DataFrame(profiles)['trips'].to_list()

In [ ]:
trips[1]

In [ ]:
trips_all = []

for trip in trips:
  trips_all.extend(trip)

In [ ]:
trips_all = []

for trip in trips:
  if type(trip) != float:
    trips_all.extend(trip)

In [ ]:
pd.DataFrame(trips_all)

## Introducing NoSQL with MongoDB
What if there was a Database where we could insert all these documents?
Well, there is one and it's called MongoDB and is powering many many of the apps that you use daily...

you can install it on your machine, on uCloud or use the free-tier version https://cloud.mongodb.com/ from mongodb.com (Atlas)

Learn more with [this course](https://campus.datacamp.com/courses/introduction-to-using-mongodb-for-data-science-with-python/)

In [ ]:
# pymongo will make it possible for our python driver to connect (pymongo is the official python driver)
import pymongo

In [ ]:
from pymongo import MongoClient
client = MongoClient('xxx')

In [ ]:
db = client.test_database


In [ ]:
collection = db.test_collection

In [ ]:
collection.insert_one(profiles[0])

In [ ]:
profiles[0]

In [ ]:
collection.find_one()

In [ ]:
profiles[0]

In [ ]:
collection.insert_many(profiles[1:])

In [ ]:
a = collection.find({'trips.country_code' : 'DK' })

In [ ]:
collection.count_documents({'trips.country_code' : 'DK' })

In [ ]:
list(a)

### More complex aggregation pipeline

Mongo allows us to perform really nice data manipulation within the DB (superfast)
However, some slightly different syntax is needed. (Not beginner friendly...but doable)

In [ ]:
# aggregation

c = collection.aggregate([
   {'$match':{'trips.country_code' : 'DK' }}, # Find all trips that went to DK
   {'$project': 
   {'trips': 1} #project (reveal) only data from trips key
   }, 
   {'$unwind': '$trips'} # unwind...flatten
  ])

In [ ]:
# turning cursor into list
c = list(c)

In [ ]:
# flatten using list comprehension
c = [c['trips'] for c in c]

In [ ]:
# making it a DF for in memory analysis
pd.DataFrame(c)

#### sequence matters

In [ ]:
# aggregation

c = collection.aggregate([
   
  { '$project': 
   {'trips': 1} #project (reveal) only data from trips key
   }, 
   {'$unwind': '$trips'}, # unwind...flatten
   {'$match':{'trips.country_code' : 'DK' }} # Find all trips that went to DK
  ])

In [ ]:
# turning cursor into list
c = list(c)

In [ ]:
# flatten using list comprehension
c = [c['trips'] for c in c]

In [ ]:
# making it a DF for in memory analysis
pd.DataFrame(c)

### Adding some EDA viz to it

In [ ]:
# aggregation

c = collection.aggregate([
  { '$project': 
   {'trips': 1} 
   }, 
   {'$unwind': '$trips'}
  
  ])

In [ ]:

c = list(c)

In [ ]:

trips_df = pd.DataFrame([x['trips'] for x in c])

In [ ]:
trips_df

In [ ]:
pd.to_datetime(trips_df.epoch_start, unit='s')

In [ ]:
trips_df.epoch_start = pd.to_datetime( trips_df.epoch_start, unit='s')
trips_df.epoch_end = pd.to_datetime( trips_df.epoch_end, unit='s')

In [ ]:
trips_df['length'] = pd.to_timedelta(trips_df.epoch_duration, unit='s')

In [ ]:
trips_df.place.value_counts()[:20]

In [ ]:
trips_df.groupby('place')['length'].median().nlargest(20)

In [ ]:
trips_2020 = trips_df[(trips_df.epoch_start > pd.to_datetime('2020')) & (trips_df.epoch_end < pd.to_datetime('2021'))]

In [ ]:
trips_2020.length.mean()

In [ ]:
len(trips_2020)

In [ ]:
trips_2021 = trips_df[(trips_df.epoch_start > pd.to_datetime('2021')) & (trips_df.epoch_end < pd.to_datetime('2022'))]

In [ ]:
trips_2021.length.mean()

In [ ]:
len(trips_2021)

In [ ]:
pd.Series([x.year for x in trips_df.epoch_start]).hist()

## Introducing Altair

https://altair-viz.github.io/
The probably best open source interactive plotting library for Python.

In [ ]:
trips_df.groupby('place')['length'].median().nlargest(20)

In [ ]:
import altair as alt

In [ ]:
source = pd.DataFrame(trips_df.groupby('place')['length'].median().nlargest(20)).reset_index()

source['length'] = [d.days for d in source['length']]

alt.Chart(source).mark_bar().encode(
    x='length',
    y='place:N',
    tooltip=['length']
).interactive()